# Download dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown 17FfAoAsvEhTFaQXB97xoVVKN4rDeBE41
!gdown 1b4Z_ngrkzTxZ1U1bWqCfE9gERvRZCYqq

!unzip '/content/Train_144.mat.zip'
!unzip '/content/Test_144.mat.zip'
!mkdir 'test'
!mkdir 'train'
!mv '/content/Test_144.mat' '/content/test/Test_144.mat'
!mv '/content/Train_144.mat' '/content/train/Train_144.mat'

Downloading...
From: https://drive.google.com/uc?id=17FfAoAsvEhTFaQXB97xoVVKN4rDeBE41
To: /content/Train_144.mat.zip
100% 2.48G/2.48G [00:35<00:00, 69.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1b4Z_ngrkzTxZ1U1bWqCfE9gERvRZCYqq
To: /content/Test_144.mat.zip
100% 355M/355M [00:05<00:00, 59.9MB/s]
Archive:  /content/Train_144.mat.zip
  inflating: Train_144.mat           
Archive:  /content/Test_144.mat.zip
  inflating: Test_144.mat            


In [ ]:
!gdown 1Vf4pJJSU8uUJOVnBqQGogB0Fi9Ll_Yt3
!gdown 1RQ8fj1UfUzroI1g_jmgTMbT3W5teSMZB

!unzip '/content/Train_16_144.mat.zip'
!unzip '/content/mmWave_Test.mat.zip'
!mkdir 'test'
!mkdir 'train'
!mv '/content/mmWave_Test.mat' '/content/test/Test_16_144.mat'
!mv '/content/Train_16_144.mat' '/content/train/Train_16_144.mat'

# Preprations

In [ ]:
!pip install mat73
import torch
from tabulate import tabulate
from scipy.io import savemat
import numpy as np
import torch.nn as nn
import torchvision
import time
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import sys
import random
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy.linalg as LA
import os
import math
import mat73
from scipy.interpolate import Rbf
import scipy.io as sio
from numpy import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Pre-Processing

In [ ]:
# DFT matrix
def DFT_matrix(N):
    m, n = np.meshgrid(np.arange(N), np.arange(N))
    omega = np.exp( - 2 * np.pi * 1j / N )
    D = np.power( omega, m * n )
    return D


# Create a function to interpolate using Rbf
def interpolate(matrix):
    rows, cols = np.nonzero(matrix)
    values = matrix[rows, cols]
    grid_x, grid_y = np.meshgrid(np.arange(matrix.shape[1]), np.arange(matrix.shape[0]))
    rbf = Rbf(cols, rows, values, function='gaussian')
    interpolated_matrix = rbf(grid_x, grid_y)
    return interpolated_matrix

In [ ]:
M_t = 36
Nt = 16
M = 144
ch = 3

snr = 20
SNR=10.0**(snr/10.0) # transmit power

IRS_index = range(0,M,4)

PHI = np.zeros((M,M_t), dtype=complex)
PHI_t = DFT_matrix(len(IRS_index))

for c in range(M_t):
  index = IRS_index[c]
  for i in range(M_t):
    PHI[index,i]=PHI_t[c,i]
print(M_t)
print(len(IRS_index))

phi = np.dot(PHI,np.transpose(np.conjugate(PHI)))

36
36


In [ ]:
############## training set generation ##################
file_dir = '/content/train'

data_num_train = 40000
data_num_file = 40000
Y=np.zeros((Nt,M_t), dtype=float)
H_train=np.zeros((data_num_train,Nt,M,ch), dtype=float)
H_train_noisy=np.zeros((data_num_train,Nt,M,ch), dtype=float)
filedir = os.listdir(file_dir)  # type the path of training data
SNR_factor = 31.1358
SNRr = 0

for filename in filedir:
    newname = os.path.join(file_dir, filename)
    data = mat73.loadmat(newname)
    channel = data['H']
    # data = []
    for i in range(data_num_file):
        H=channel[:,:,i]
        H_re = np.real(H)
        H_im = np.imag(H)
        H_ab = np.abs(H)
        H_train[i, :, :, 0] = H_re
        H_train[i, :, :, 1] = H_im
        H_train[i, :, :, 2] = H_ab

        N = np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M)) + 1j * np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M))
        N_hat = np.dot(N,phi)

        H_hat = np.dot(H,phi)
        Y = H_hat + 1.0 / np.sqrt(SNR_factor*SNR) * N_hat
        H_hat_re = np.real(Y)
        H_hat_im = np.imag(Y)
        H_hat_ab = np.abs(Y)
        SNRr = SNRr + SNR_factor*SNR * (LA.norm(H_hat)) ** 2 / (LA.norm(N_hat)) ** 2

        H_train_noisy[i, :, :, 0] = H_hat_re
        H_train_noisy[i, :, :, 1] = H_hat_im
        H_train_noisy[i, :, :, 2] = H_hat_ab

print(SNRr/(data_num_train))
SNRR = 10 * np.log10(SNRr/(data_num_train))
print(SNRR)
print(H_train.shape,H_train_noisy.shape)
# channel = []

100.19660318333281
20.00852998539806
(40000, 16, 144, 3) (40000, 16, 144, 3)


# Dataset and prepration

In [ ]:
class mydataset(Dataset):
  def __init__(self , datas , labels):
    super(mydataset,self).__init__()
    self.datas = torch.tensor(datas , dtype=torch.float)
    self.datas = torch.permute(self.datas, (0,3,1,2)) # must to use torch.permute beacuse the channel must be after batch size [batch_size , channel , nr , nt]

    self.labels = torch.tensor(labels , dtype=torch.float)
    self.labels = torch.permute(self.labels, (0,3,1,2)) #  must to use torch.permute beacuse the channel must be after batch size [batch_size , channel , nr , nt]
  def __getitem__(self, index):
    return self.datas[index] , self.labels[index]
  def __len__(self):
    return self.datas.shape[0]

In [ ]:
batch_size = 64

trainset = mydataset(H_train_noisy , H_train)
# trainset,valset = train_test_split(trainset,test_size=0.2,shuffle=True,random_state=123)

trainloader = DataLoader(trainset , batch_size = batch_size , shuffle = True, drop_last=True)
# valloader = DataLoader(valset , batch_size = batch_size , shuffle = True, drop_last=True)

In [ ]:
batch_test=10
trainset2 = mydataset(H_train_noisy , H_train)
trainloader2 = DataLoader(trainset2 , batch_size = batch_test , shuffle = True, drop_last=True)

def train(testloader,model,size):
  nmse=np.zeros((size,1), dtype=float16)
  k = 0

  model.eval()
  for (input,label) in tqdm(testloader):
    nmse2=torch.zeros((batch_test,1), dtype=torch.float32)

    input,label = input.to(device),label.to(device)
    decoded_channel = model(input)

    for n in range(batch_test):
        MSE=((label[n,:,:,:]-decoded_channel[n,:,:,:])**2).sum()
        norm_real=((label[n,:,:,:])**2).sum()
        nmse2[n]=MSE/norm_real

    a = nmse2.sum()
    nmse[k] = a.detach().numpy()
    k = k + 1

  return nmse

In [ ]:
H_train_noisy =[]
H_train = []
# data = []
# channel = []
# x=trainset.__getitem__(3)[0]
# type(x)

# NMSE Valid

In [ ]:
############## training set generation ##################
file_dir = '/content/train'

data_num_train = 10000
data_num_file = 10000
Y=np.zeros((Nt,M_t), dtype=float)
H_valid=np.zeros((data_num_train,Nt,M,ch), dtype=float)
H_valid_noisy=np.zeros((data_num_train,Nt,M,ch), dtype=float)
filedir = os.listdir(file_dir)  # type the path of training data
SNR_factor = 31.1358
SNRr = 0

for filename in filedir:
    newname = os.path.join(file_dir, filename)
    # data = mat73.loadmat(newname)
    channel = data['H']
    # data = []
    for i in range(40000,50000):
        H=channel[:,:,i]
        H_re = np.real(H)
        H_im = np.imag(H)
        H_ab = np.abs(H)
        H_valid[i-40000, :, :, 0] = H_re
        H_valid[i-40000, :, :, 1] = H_im
        H_valid[i-40000, :, :, 2] = H_ab

        N = np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M)) + 1j * np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M))
        N_hat = np.dot(N,phi)

        H_hat = np.dot(H,phi)
        Y = H_hat + 1.0 / np.sqrt(SNR_factor*SNR) * N_hat
        H_hat_re = np.real(Y)
        H_hat_im = np.imag(Y)
        H_hat_ab = np.abs(Y)
        SNRr = SNRr + SNR_factor*SNR * (LA.norm(H_hat)) ** 2 / (LA.norm(N_hat)) ** 2

        H_valid_noisy[i-40000, :, :, 0] = H_hat_re
        H_valid_noisy[i-40000, :, :, 1] = H_hat_im
        H_valid_noisy[i-40000, :, :, 2] = H_hat_ab

print(SNRr/(data_num_train))
SNRR = 10 * np.log10(SNRr/(data_num_train))
print(SNRR)
print(H_valid.shape,H_valid_noisy.shape)
channel = []

100.58000745257286
20.025116634643787
(10000, 16, 144, 3) (10000, 16, 144, 3)


In [ ]:
batch_test=10
validset = mydataset(H_valid_noisy , H_valid)
validloader = DataLoader(validset , batch_size = batch_test , shuffle = True, drop_last=True)

def valid(testloader,model,size):
  nmse=np.zeros((size,1), dtype=float16)
  k = 0

  model.eval()
  for (input,label) in tqdm(testloader):
    nmse2=torch.zeros((batch_test,1), dtype=torch.float32)

    input,label = input.to(device),label.to(device)
    t1=time.time()
    decoded_channel = model(input)

    t2=time.time()

    for n in range(batch_test):
        MSE=((label[n,:,:,:]-decoded_channel[n,:,:,:])**2).sum()
        norm_real=((label[n,:,:,:])**2).sum()
        nmse2[n]=MSE/norm_real

    a = nmse2.sum()
    nmse[k] = a.detach().numpy()
    k = k + 1

  return nmse

In [ ]:
H_valid = []
H_valid_noisy = []
data = []
channel = []
# x = trainset.__getitem__(0)

# SSRGAN

## ConvBlock

In [ ]:
class ConvBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            discriminator=False,
            use_act=True,
            use_bn=True,
            **kwargs,
    ):
        super().__init__()
        self.use_act = use_act
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs, bias=not use_bn)
        self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
        self.act = (
            nn.LeakyReLU(0.2, inplace=True)
            if discriminator
            else nn.PReLU(num_parameters=out_channels)
        )

    def forward(self, x):
        return self.act(self.bn(self.cnn(x))) if self.use_act else self.bn(self.cnn(x))



## UpsampleBlock

In [ ]:
class UpsampleBlock(nn.Module):
    def __init__(self, in_c, scale_factor):
        super().__init__()
        self.conv = nn.Conv2d(in_c, in_c * scale_factor ** 2, 3, 1, 1)
        self.ps = nn.PixelShuffle(scale_factor)  # in_c * 4, H, W --> in_c, H*2, W*2
        self.act = nn.PReLU(num_parameters=in_c)

    def forward(self, x):
        return self.act(self.ps(self.conv(x)))

## ResidualBlock

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block1 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.block2 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            use_act=False,
        )

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        return out + x


## Generator & Discriminator

In [ ]:

'''''''''''''''''''''''''''''''''''''''''''Generator'''''''''''''''''''''''''''''''''''''''''''
class Generator(nn.Module):
    def __init__(self, in_channels=ch, num_channels=64, num_blocks = 2):
        super().__init__()
        self.initial = ConvBlock(in_channels, num_channels, kernel_size=9, stride=1, padding=4, use_bn=False)
        self.residuals = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_blocks)])
        self.convblock = ConvBlock(num_channels, num_channels, kernel_size=3, stride=1, padding=1, use_act=False)
        self.upsamples = nn.Sequential(UpsampleBlock(num_channels, 1), UpsampleBlock(num_channels, 1))
        self.final = nn.Conv2d(num_channels, in_channels, kernel_size=9,  padding='same')

    def forward(self, x):
        initial = self.initial(x)
        x = self.residuals(initial)
        x = self.convblock(x) + initial
        x = self.upsamples(x)
        return torch.tanh(self.final(x))

'''''''''''''''''''''''''''''''''''''''''Discriminator'''''''''''''''''''''''''''''''''''''''''
class Discriminator(nn.Module):
    def __init__(self, in_channels=ch, features=[64, 128, 256, 512]):
        super().__init__()
        blocks = []
        for idx, feature in enumerate(features):
            blocks.append(
                ConvBlock(
                    in_channels,
                    feature,
                    kernel_size=3,
                    stride=1 + idx % 2,
                    padding=1,
                    discriminator=True,
                    use_act=True,
                    use_bn=False if idx == 0 else True,
                )
            )
            in_channels = feature

        self.blocks = nn.Sequential(*blocks)

        self.conv = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=32,kernel_size=1),
                                  nn.ReLU(),
                                  nn.ConvTranspose2d(in_channels=32, out_channels=ch, kernel_size=4,stride=4),
                                  )

        self.linear = nn.Sequential(

            nn.Flatten()

        )

    def forward(self, x):
        x = self.blocks(x)
        x = self.conv(x)
        x = self.linear(x)

        x = torch.reshape(x, (batch_size, ch, Nt, M))
        return x



In [ ]:
gen = Generator(in_channels=ch).to(device)
disc = Discriminator(in_channels=ch).to(device)
gen

Generator(
  (initial): ConvBlock(
    (cnn): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (bn): Identity()
    (act): PReLU(num_parameters=64)
  )
  (residuals): Sequential(
    (0): ResidualBlock(
      (block1): ConvBlock(
        (cnn): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): PReLU(num_parameters=64)
      )
      (block2): ConvBlock(
        (cnn): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): PReLU(num_parameters=64)
      )
    )
    (1): ResidualBlock(
      (block1): ConvBlock(
        (cnn): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (

In [ ]:
x = torch.rand(64, ch, 16, 144).to(device)
s = gen(x).shape
print(s)

torch.Size([64, 3, 16, 144])


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The gen has {count_parameters(gen):,} trainable parameters')
print(f'The disc has {count_parameters(disc):,} trainable parameters')


The gen has 290,499 trainable parameters
The disc has 1,569,827 trainable parameters


<ipython-input-16-b70f27d26e37>:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Default setting

In [ ]:
LEARNING_RATE=0.0002
opt_gen = torch.optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))

lr_scheduler_gen = torch.optim.lr_scheduler.StepLR(optimizer = opt_gen , step_size = 20 , gamma=0.6)
lr_scheduler_disc = torch.optim.lr_scheduler.StepLR(optimizer = opt_disc , step_size = 20 , gamma=0.6)

mse = nn.MSELoss()
criterion = nn.MSELoss()

In [ ]:
'''
  this class is for calculating the final loss of each epoch
'''
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger('SRGAN: ')

# Train

In [ ]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # # If we don't do this then it will just have learning rate of old checkpoint
    # # and it will lead to many hours of debugging \:
    # for param_group in optimizer.param_groups:
    #     param_group["lr"] = lr

In [ ]:
discloss = []
genloss = []
def train_fn(loader, disc, gen, opt_gen, opt_disc, mse):
    gen.train()
    disc.train()

    loop = tqdm(loader, leave=True)
    total_loss_gen = AverageMeter()
    total_loss_dis = AverageMeter()

    for (HLR, HSR) in loop:
        HSR = HSR.to(device)
        HLR = HLR.to(device)

        HSRhat = gen(HLR)
        disc_real = disc(HSR)
        disc_fake = disc(HSRhat.detach())
        disc_loss_real = mse(disc_real, torch.ones_like(disc_real) - 0.1 * torch.rand_like(disc_real))
        disc_loss_fake = mse(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = disc_loss_fake + disc_loss_real
        total_loss_dis.update(loss_disc)

        opt_disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        disc_fake = disc(HSRhat)
        adversarial_loss = 1e-3 * mse(disc_fake, torch.ones_like(disc_fake))
        loss_for_gen = criterion(HSRhat, HSR)
        gen_loss = loss_for_gen + adversarial_loss
        total_loss_gen.update(gen_loss)

        opt_gen.zero_grad()
        gen_loss.backward()
        opt_gen.step()

#################################################### SAVING GENERATOR AND DISCRIMINATOR ################################################
    if epoch % 5 == 0 :
      path_gen = 'Generator_epoch_{}_{}dB.pth.tar'.format(epoch,snr)
      path_disc = 'Discrimnator_epoch_{}_{}dB.pth.tar'.format(epoch,snr)

      save_checkpoint(gen, opt_gen, filename=path_gen)
      save_checkpoint(disc, opt_disc, filename=path_disc)


    discloss.append(total_loss_dis.avg.cpu().detach().numpy())
    genloss.append(total_loss_gen.avg.cpu().detach().numpy())

    logger.info(f'Train: Epoch:{epoch} \t Loss_Generator:{total_loss_gen.avg:.4} \t lr:{lr_scheduler_gen.get_last_lr()}')
    logger.info(f'Train: Epoch:{epoch} \t Loss_Discrimnator:{total_loss_dis.avg:.4} \t lr:{lr_scheduler_disc.get_last_lr()}')

# Valid

In [ ]:
def validate_fn(val_loader, gen, disc, mse, bce):
    gen.eval()
    disc.eval()
    total_loss_gen = AverageMeter()
    total_loss_dis = AverageMeter()

    with torch.no_grad():
        for (x, label) in val_loader:
            label = label.to(device)
            x = x.to(device)

            fake = gen(x)
            disc_real = disc(label)
            disc_fake = disc(fake)

            disc_loss_real = bce(disc_real, torch.ones_like(disc_real) - 0.1 * torch.rand_like(disc_real))
            disc_loss_fake = bce(disc_fake, torch.zeros_like(disc_fake))
            loss_disc = disc_loss_fake + disc_loss_real
            total_loss_dis.update(loss_disc.item())

            disc_fake = disc(fake)
            adversarial_loss = 1e-3 * bce(disc_fake, torch.ones_like(disc_fake))
            loss_for_vgg = mse(fake, label)
            gen_loss = loss_for_vgg + adversarial_loss
            total_loss_gen.update(gen_loss.item())

    gen.train()
    disc.train()

    avg_val_gen_loss = total_loss_gen.avg
    avg_val_dis_loss = total_loss_dis.avg

    return avg_val_gen_loss, avg_val_dis_loss


# Training phase

In [ ]:
# load_gen= '/content/Generator_epoch_50.pth.tar'
# load_disc='/content/Discrimnator_epoch_50.pth.tar'
# load_checkpoint(load_gen,gen,opt_gen,LEARNING_RATE)
# load_checkpoint(load_disc, disc, opt_disc,LEARNING_RATE)

NMSE_valid = []
NMSE_train = []
start , end = 1 , 101
for epoch in range (start , end):
  train_fn(trainloader, disc, gen, opt_gen, opt_disc, mse)
  nmse_valid = valid(validloader,gen,1000)
  nmse_train = train(trainloader2,gen,4000)
  a = nmse_train.sum()/(40000)
  b = nmse_valid.sum()/(10000)
  NMSE_valid.append(b)
  NMSE_train.append(a)
  print('NMSE_valid = {}'.format(b))
  print('NMSE_train = {}'.format(a))
  print('--------------------------------------------------------------')
  lr_scheduler_gen.step()
  lr_scheduler_disc.step()

  0%|          | 3/625 [01:03<3:38:03, 21.03s/it]


KeyboardInterrupt: ignored

In [ ]:
plt.semilogy(NMSE_valid)
plt.semilogy(NMSE_train)
plt.show()
# np.min(NMSE_test)

In [ ]:
from scipy.io import savemat
import numpy as np

mdic = {"NMSE_test{}".format(snr): NMSE_test, "label": "experiment"}
mdic
{'a':NMSE_test,'label': 'experiment'}
savemat("NMSE_test{}.mat".format(snr), mdic)

In [ ]:
print('generator',genloss[-1])
print('discrimnator',discloss[-1])

fig2, (generator,semilog_gen) = plt.subplots(1,2, figsize=(15, 2))

plt.grid()
generator.plot(genloss,label="objectloss",linestyle='-',color='green')
plt.xlabel("epoch")
plt.ylabel("Loss")

print()
fig, (discrimnator,semilog_disc) = plt.subplots(1,2, figsize=(15, 2))

plt.grid()
discrimnator.plot(discloss,label="objectloss",color='blue')
plt.xlabel("epoch")
plt.ylabel("Loss")

# Test

In [ ]:
# SNR=10.0**(snr/10.0) # transmit power
############## training set generation ##################
file_dir_test = '/content/test'

data_num_test = 1000
data_num_file = 1000
Y=np.zeros((Nt,M), dtype=float)
H_test=np.zeros((data_num_test,Nt,M,ch), dtype=float)
H_test_noisy=np.zeros((data_num_test,Nt,M,ch), dtype=float)
filedir = os.listdir(file_dir_test)  # type the path of training data

# SNR_factor = 11.5
SNR_factor = 31.1358
SNRr = 0
n=0
for filename in filedir:
    newname = os.path.join(file_dir_test, filename)
    data = mat73.loadmat(newname)
    # data = sio.loadmat(newname)
    channel = data['H']
    for i in range(data_num_test):
        H=channel[:,:,i]
        H_re = np.real(H)
        H_im = np.imag(H)
        H_ab = np.abs(H)
        H_test[i, :, :, 0] = H_re
        H_test[i, :, :, 1] = H_im
        H_test[i, :, :, 2] = H_ab
        phi = np.dot(PHI,np.transpose(np.conjugate(PHI)))
        N = np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M)) + 1j * np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M))

        N_hat = np.dot(N,phi)
        H_hat = np.dot(H,phi)

        Y = H_hat + 1.0 / np.sqrt(SNR_factor*SNR) * N_hat
        H_hat_re = np.real(Y)
        H_hat_im = np.imag(Y)
        H_hat_ab = np.abs(Y)
        SNRr = SNRr + SNR_factor*SNR * (LA.norm(H_hat)) ** 2 / (LA.norm(N_hat)) ** 2

        H_test_noisy[i, :, : ,0] = H_hat_re
        H_test_noisy[i, :, :, 1] = H_hat_im
        H_test_noisy[i, :, :, 2] = H_hat_ab
    n=n+1
print(SNRr/(data_num_test))
SNRR = 10 * np.log10(SNRr/(data_num_test))
print(SNRR)
print(H_test.shape,H_test_noisy.shape)

98.58847683026409
19.938261569140032
(1000, 16, 144, 3) (1000, 16, 144, 3)


In [ ]:
batch_test=10
testset = mydataset(H_test_noisy , H_test)
testloader = DataLoader(testset , batch_size = batch_test , shuffle = False, drop_last=True)

def test(testloader,model):
  nmse=np.zeros((1000,1), dtype=float16)
  k = 0
  model.eval()
  for (input,label) in tqdm(testloader):
    nmse2=torch.zeros((batch_test,1), dtype=torch.float16)

    input,label = input.to(device),label.to(device)
    t1=time.time()
    decoded_channel = model(input)
    t2=time.time()

    for n in range(batch_test):
        MSE=((label[n,:,:,:]-decoded_channel[n,:,:,:])**2).sum()
        norm_real=((label[n,:,:,:])**2).sum()
        nmse2[n]=MSE/norm_real

    a = nmse2.sum()
    nmse[k] = a.detach().numpy()
    k = k + 1

  return nmse
H_test = []
H_test_noisy = []
data = []
channel = []

# Least square & NMSE

In [ ]:
load_gen= '/content/Generator_epoch_80_20dB.pth.tar'
load_checkpoint(load_gen,gen,opt_gen,LEARNING_RATE)

=> Loading checkpoint


In [ ]:
batch_test=10
testset = mydataset(H_test_noisy , H_test)
testloader = DataLoader(testset , batch_size = batch_test , shuffle = False, drop_last=True)

def test(testloader,model):
  nmse=np.zeros((1000,1), dtype=float16)
  k = 0
  model.eval()
  for (input,label) in tqdm(testloader):
    nmse2=torch.zeros((batch_test,1), dtype=torch.float16)

    input,label = input.to(device),label.to(device)
    t1=time.time()
    decoded_channel = model(input)
    t2=time.time()

    for n in range(batch_test):
        MSE=((label[n,:,:,:]-decoded_channel[n,:,:,:])**2).sum()
        norm_real=((label[n,:,:,:])**2).sum()
        nmse2[n]=MSE/norm_real

    a = nmse2.sum()
    nmse[k] = a.detach().numpy()
    k = k + 1

  return nmse
H_test = []
H_test_noisy = []
data = []
channel = []

In [ ]:
nmse = test(testloader,gen)
nmse.sum()/(1000)

100%|██████████| 10/10 [00:00<00:00, 55.66it/s]


0.04975

In [ ]:
H_test=(torch.permute(torch.tensor(H_test , dtype = torch.float32),(0,3,1,2))).to(device)
H_test_noisy=torch.permute(torch.tensor(H_test_noisy,dtype = torch.float32),(0,3,1,2)).to(device)

In [ ]:
import time
t1=time.time()
# dn_model.eval()
gen.eval()
decoded_channel = gen(H_test_noisy)
# decoded_channel = dn_model(H_test_interpolated)
t2=time.time()
nmse2=torch.zeros((data_num_test,1), dtype=torch.float16)
for n in range(data_num_test):
    MSE=((H_test[n,:,:,:]-decoded_channel[n,:,:,:])**2).sum()
    norm_real=((H_test[n,:,:,:])**2).sum()
    nmse2[n]=MSE/norm_real
print('NMSE = ',nmse2.sum()/(data_num_test))  # calculate NMSE after training stage (testing performance)

print('t2-t1 = ',t2-t1)

NMSE =  tensor(1.0156, dtype=torch.float16, grad_fn=<DivBackward0>)
t2-t1 =  0.011760711669921875


In [ ]:
from scipy.io import savemat
import numpy as np
decoded_channel = torch.permute(decoded_channel , (2,3,1,0))
decoded_channel = decoded_channel.cpu()
x= decoded_channel.detach().numpy()
mdic = {"H_SRGAN_{}".format(snr): x, "label": "experiment"}
mdic
{'a':x,'label': 'experiment'}
savemat("H_SRGAN_{}.mat".format(snr), mdic)
# !mv '/content/F_RF2.mat' '/content/drive/MyDrive/Colab Notebooks/ML+CE/Dataset/F_RF'

# To Save Estimated Channel


In [ ]:
# SNR=10.0**(snr/10.0) # transmit power
############## training set generation ##################
file_dir_test = '/content/test'

data_num_test = 10000
data_num_file = 10000
Y=np.zeros((Nt,M), dtype=float)
H_test=np.zeros((data_num_test,Nt,M,ch), dtype=float)
H_test_noisy=np.zeros((data_num_test,Nt,M,ch), dtype=float)
filedir = os.listdir(file_dir_test)  # type the path of training data

SNR_factor = 31.1358
SNRr = 0
n=0
for filename in filedir:
    newname = os.path.join(file_dir_test, filename)
    data = mat73.loadmat(newname)
    # data = sio.loadmat(newname)
    channel = data['H']
    for i in range(data_num_test):
        H=channel[:,:,i]
        H_re = np.real(H)
        H_im = np.imag(H)
        H_ab = np.abs(H)
        H_test[n*data_num_file+i, :, :, 0]=H_re
        H_test[n*data_num_file+i, :, :, 1] = H_im
        # H_test[n*data_num_file+i, :, :, 2] = H_ab
        phi = np.dot(PHI,np.transpose(np.conjugate(PHI)))
        N = np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M)) + 1j * np.random.normal(0, 1 / np.sqrt(2), size=(Nt, M))
        N_hat = np.dot(N,phi)

        H_hat = np.dot(H,phi)
        Y = H_hat + 1.0 / np.sqrt(SNR_factor*SNR) * N_hat
        H_hat_re = np.real(Y)
        H_hat_im = np.imag(Y)
        # H_hat_ab = np.abs(Y)
        SNRr = SNRr + SNR_factor*SNR * (LA.norm(H_hat)) ** 2 / (LA.norm(N_hat)) ** 2

        H_test_noisy[n*data_num_file+i, :, : ,0]=H_hat_re
        H_test_noisy[n*data_num_file+i, :, :, 1] = H_hat_im
        # H_test_noisy[n*data_num_file+i, :, :, 2] = H_hat_ab
    n=n+1
print(SNRr/(data_num_test))
SNRR = 10 * np.log10(SNRr/(data_num_test))
print(SNRR)
print(H_test.shape,H_test_noisy.shape)
# index3 = np.where(abs(H_test) > 1)

3.1680845891911114
5.007967689176081
(10000, 16, 144, 2) (10000, 16, 144, 2)


In [ ]:
batch_test=10
testset = mydataset(H_test_noisy , H_test)
testloader = DataLoader(testset , batch_size = batch_test , shuffle = False, drop_last=True)

H_new = []
def test(testloader,model):
  nmse=np.zeros((1000,1), dtype=float16)
  k = 0
  model.eval()
  for (input,label) in tqdm(testloader):
    nmse2=torch.zeros((batch_test,1), dtype=torch.float16)

    input,label = input.to(device),label.to(device)
    t1=time.time()
    decoded_channel = model(input)
    t2=time.time()

    for n in range(batch_test):
        MSE=((label[n,:,:,:]-decoded_channel[n,:,:,:])**2).sum()
        norm_real=((label[n,:,:,:])**2).sum()
        nmse2[n]=MSE/norm_real

    a = nmse2.sum()
    nmse[k] = a.detach().numpy()
    k = k + 1
    ALI = torch.permute(decoded_channel , (0,2,3,1))
    H_new.append(ALI.cpu().detach().numpy())
    # print('NMSE = ',nmse2.sum()/10)  # calculate NMSE after training stage (testing performance)
  return nmse
# H_test = []
# H_test_noisy = []
# data = []
# channel = []

In [ ]:
load_gen= '/content/Generator_SNR_-10dB_10.pth.tar'
load_checkpoint(load_gen,gen,opt_gen,LEARNING_RATE)

In [ ]:
nmse = test(testloader,gen)
nmse.sum()/(10000)

100%|██████████| 1000/1000 [00:06<00:00, 157.12it/s]


0.0158625

In [ ]:
H_SRGAN20 = np.zeros((data_num_test,Nt,M,ch), dtype=float)
m = 0
for i in range(1000):
  a = H_new[i]
  for j in range(10):
    H_SRGAN20[j+m,:,:,:] = a[j]
  m+= 10

In [ ]:
nmse2=torch.zeros((data_num_test,1), dtype=torch.float32)
for n in range(data_num_test):
    MSE=((H_test[n,:,:,:]-H_SRGAN20[n,:,:,:])**2).sum()
    norm_real=((H_test[n,:,:,:])**2).sum()
    nmse2[n]=MSE/norm_real
print('NMSE = ',nmse2.sum()/(data_num_test))

NMSE =  tensor(0.0159)


In [ ]:
H_SRGAN5 = torch.tensor(H_SRGAN20)
H_SRGAN5 = torch.permute(H_SRGAN5 , (1,2,3,0))
H_SRGAN5 = H_SRGAN5.cpu().detach().numpy()

In [ ]:
from scipy.io import savemat
import numpy as np

mdic = {"H_SRGAN5": H_SRGAN5, "label": "experiment"}
mdic
{'a':H_SRGAN5,'label': 'experiment'}
savemat("H_SRGAN5.mat", mdic)

In [ ]:
!mv '/content/H_SRGAN5.mat' '/content/drive/MyDrive/RESULT CE'